In [1]:
from datetime import datetime

In [2]:
current = datetime.utcnow()

In [3]:
current

datetime.datetime(2021, 2, 20, 23, 24, 47, 755283)

In [4]:
import json

In [5]:
json.dumps(current)

TypeError: Object of type datetime is not JSON serializable

`YYYY-MM-DDTHH:MM:SS`

In [6]:
str(current)

'2021-02-20 23:24:47.755283'

In [7]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

In [8]:
format_iso(current)

'2021-02-20T23:24:47'

In [9]:
current.isoformat()

'2021-02-20T23:24:47.755283'

In [10]:
log_record = {'time': datetime.utcnow().isoformat(),
             'message': 'testing'}

In [11]:
print(json.dumps(log_record, indent=2))

{
  "time": "2021-02-20T23:31:49.605958",
  "message": "testing"
}


In [12]:
log_record = {'time': format_iso(datetime.utcnow()),
             'message': 'testing'}

In [13]:
print(json.dumps(log_record, indent=2))

{
  "time": "2021-02-20T23:32:35",
  "message": "testing"
}


In [14]:
log_record = {'time': datetime.utcnow(),
             'message': 'testing'}

In [15]:
json.dumps(log_record, default=format_iso)

'{"time": "2021-02-20T23:34:02", "message": "testing"}'

In [16]:
def format_general(arg):
    return 'Unknown serialization'

In [17]:
json.dumps(log_record, default=format_general)

'{"time": "Unknown serialization", "message": "testing"}'

In [18]:
log_record = {'time': datetime.utcnow(),
             'message': 'testing',
             'args': {10, "test"}}

In [19]:
json.dumps(log_record, default=format_general)

'{"time": "Unknown serialization", "message": "testing", "args": "Unknown serialization"}'

In [22]:
log_record = {
    'time1': datetime.utcnow(),
    'time2': datetime.utcnow(),
    'message': 'Testing...',
    'args': {1, 2, 3}
}

In [23]:
json.dumps(log_record, default=format_iso)

AttributeError: 'set' object has no attribute 'strftime'

In [24]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)

In [25]:
json.dumps(log_record, default=custom_json_formatter)

'{"time1": "2021-02-20T23:37:42.067759", "time2": "2021-02-20T23:37:42.067759", "message": "Testing...", "args": [1, 2, 3]}'

In [26]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt.isoformat()
        }

In [27]:
p = Person('John', 82)

In [28]:
p.toJSON()

{'name': 'John', 'age': 82, 'create_dt': '2021-02-20T23:41:47.286503'}

In [29]:
log_record = dict(time=datetime.utcnow(),
                 message='Created new person record.',
                 person=p
                 )

In [30]:
log_record

{'time': datetime.datetime(2021, 2, 20, 23, 42, 59, 223807),
 'message': 'Created new person record.',
 'person': Person(name=John, age=82)}

In [31]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-02-20T23:42:59.223807", "message": "Created new person record.", "person": null}'

In [32]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()

In [33]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-02-20T23:42:59.223807", "message": "Created new person record.", "person": {"name": "John", "age": 82, "create_dt": "2021-02-20T23:41:47.286503"}}'

In [40]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()
        
    def __repr__(self):
        return f'Person(name={self.name}, age={self.age})'
    
    def toJSON(self):
        return vars(self)

In [41]:
p = Person('Python', 45)

In [42]:
p

Person(name=Python, age=45)

In [43]:
log_record = dict(time=datetime.utcnow(),
                 message='Created new person record.',
                 person=p
                 )

In [44]:
log_record

{'time': datetime.datetime(2021, 2, 20, 23, 47, 14, 439060),
 'message': 'Created new person record.',
 'person': Person(name=Python, age=45)}

In [45]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-02-20T23:47:14.439060", "message": "Created new person record.", "person": {"name": "Python", "age": 45, "create_dt": "2021-02-20T23:47:13.727277"}}'

In [46]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [47]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    else:
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)

In [50]:
from decimal import Decimal

pt1 = Point(1, 2)
p = Person('John', 18)
pt2 = Point(Decimal('10.5'), Decimal('100.5'))
log_record = dict(time=datetime.utcnow(),
                 message='Created new point',
                 point=pt1,
                  point_2=pt2,
                 created_by=p)

In [51]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2021-02-20T23:55:33.986033", "message": "Created new point", "point": {"x": 1, "y": 2}, "point_2": {"x": "10.5", "y": "100.5"}, "created_by": {"name": "John", "age": 18, "create_dt": "2021-02-20T23:55:33.986033"}}'

In [52]:
from functools import singledispatch

In [53]:
@singledispatch
def json_format(arg):
    print(arg)
    try:
        print('\ttring to use toJSON...')
        return arg.toJSON()
    except AttributeError:
        print('\tfailed - trying to use vars...')
        try:
            return vars(arg)
        except TypeError:
            print('\tfailed - using string repr...')
            return str(arg)

In [54]:
@json_format.register(datetime)
def _(arg):
    return arg.isoformat()

In [55]:
@json_format.register(set)
def _(arg):
    return list(arg)

In [56]:
log_record = dict(time=datetime.utcnow(),
                 message='Created new point',
                 point=pt1,
                 created_by=p)

In [57]:
print(json.dumps(log_record, indent=2, default=json_format))

Point(x=1, y=2)
	tring to use toJSON...
	failed - trying to use vars...
Person(name=John, age=18)
	tring to use toJSON...
{
  "time": "2021-02-21T00:00:33.082125",
  "message": "Created new point",
  "point": {
    "x": 1,
    "y": 2
  },
  "created_by": {
    "name": "John",
    "age": 18,
    "create_dt": "2021-02-20T23:55:33.986033"
  }
}


In [58]:
from decimal import Decimal
from fractions import Fraction

In [59]:
d = dict(a=1+1j,
        b=Decimal('0.5'),
        c=Fraction(1, 3),
        p=Person('Python', 27),
        pt=Point(0, 0),
        time=datetime.utcnow()
        )

In [60]:
d

{'a': (1+1j),
 'b': Decimal('0.5'),
 'c': Fraction(1, 3),
 'p': Person(name=Python, age=27),
 'pt': Point(x=0, y=0),
 'time': datetime.datetime(2021, 2, 21, 0, 3, 1, 107470)}

In [63]:
print(json.dumps(d, indent=2, default=json_format))

(1+1j)
	tring to use toJSON...
	failed - trying to use vars...
	failed - using string repr...
1/3
	tring to use toJSON...
	failed - trying to use vars...
	failed - using string repr...
Person(name=Python, age=27)
	tring to use toJSON...
Point(x=0, y=0)
	tring to use toJSON...
	failed - trying to use vars...
{
  "a": "(1+1j)",
  "b": "Decimal(0.5)",
  "c": "1/3",
  "p": {
    "name": "Python",
    "age": 27,
    "create_dt": "2021-02-21T00:03:01.107470"
  },
  "pt": {
    "x": 0,
    "y": 0
  },
  "time": "2021-02-21T00:03:01.107470"
}


In [62]:
@json_format.register(Decimal)
def _(arg):
    return f'Decimal({str(arg)})'

In [64]:
print(json.dumps(d, indent=2, default=json_format))

(1+1j)
	tring to use toJSON...
	failed - trying to use vars...
	failed - using string repr...
1/3
	tring to use toJSON...
	failed - trying to use vars...
	failed - using string repr...
Person(name=Python, age=27)
	tring to use toJSON...
Point(x=0, y=0)
	tring to use toJSON...
	failed - trying to use vars...
{
  "a": "(1+1j)",
  "b": "Decimal(0.5)",
  "c": "1/3",
  "p": {
    "name": "Python",
    "age": 27,
    "create_dt": "2021-02-21T00:03:01.107470"
  },
  "pt": {
    "x": 0,
    "y": 0
  },
  "time": "2021-02-21T00:03:01.107470"
}
